# google-local-rating-prediction

### 1. Objective
The goal of this task is to predict a user’s star rating of a particular business. The following pipeline was
followed to implement the task :

### 2. Data-set preprocessing and preparation

#### Note : Download and unzip the dataset from : http://jmcauley.ucsd.edu/data/assignment1.tar.gz

The Google Local dataset has 200,000 reviews of businesses’ rated by users. The dataset contains many 
attributes but for the purpose of this task, only the following were used:
* businessID: The ID of the business. This is a hashed product identifier from Google.
* userID: The ID of the reviewer. This is a hashed user identifier from Google.
* rating: The star rating of the user’s review.

The dataset is divided into train, validation using k-fold ( k = 3) split. The entire 200,000 data points were used for building
the final model once all the parameters were estimated after validation

### 3. Recommender System Models Used

The approach used was to build an ensemble of 6 different recommender system algorithms to model the
relation between the users and businesses. For each algorithm the 200,000 X 3 train-set was converted into a 
pandas dataframe and then was represented as a surprise trainset object before passing it into surprise’s algorithm libraries

### 3.1 BaselineOnly
Algorithm predicting the baseline rating estimate $\hat{r}_{ui}$ for given user $u$ and item $i$.

$$\hat{r}_{ui}=b_{ui}= \mu +b_u+ b_i $$

If user $u$ is unknown, then the bias $b_u$ is assumed to be zero. The same applies for item $i$ with $b_i.$

This model tries to minimize the following regularized  square error, where $\lambda$ is the regularization parameter.

$$\sum_{r_{ui} \in R_{train}} \left(r_{ui} - (\mu + b_u + b_i)\right)^2 +
\lambda \left(b_u^2 + b_i^2 \right)$$


Two separate models are built using the "baselineOnly" idea from surprise :

**Stochastic Gradient Descent (SGD)**: The parameters for SGD are :

* ’reg’ or lambda: The regularization parameter of the cost function that is optimized
* ’learning rate’: The learning rate of SGD
* ’nepochs’: The number of iteration of the SGD procedure

**Alternating Least Squares (ALS) **: The update equation for $\mu$ is the simple average of all the all ratings centered after subtracting $b_{u}$ + $b_{i}$. The update equations for $b_u$ and $b_i$ are in the figure below, where K is the $R_{trainset}$, $\lambda_2$ = $reg_i$, and $\lambda_3$ = $reg_u,$

![](images-readme/als.png)

* ’regi’: The regularization parameter for items.
* ’regu’: The regularization parameter for users.
* ’nepochs’: The number of iteration of the ALS procedure.


### 3.2 KNNBaseline

A basic collaborative filtering algorithm taking into account a baseline rating. It also accounts for the similarity sim(u,v) between user pairs (u,v). Let $N^k_i(u)$ denote the K-user-neighbourhood of user u and people who have rated item $i$. The predicted rating $r_{ui}$ is given by :


$$\hat{r}_{ui} = b_{ui} + \frac{ \sum\limits_{v \in N^k_i(u)}
\text{sim}(u, v) \cdot (r_{vi} - b_{vi})} {\sum\limits_{v \in
N^k_i(u)} \text{sim}(u, v)}$$

The similarity between users is computed by using the pearson similarity between user u and v $\in$ $N^k_i(u)$ for parameter K and item i. These sim(u,v) values are i,j positions in similarity Matrix R for the ratings/ utility matrix C, as in figure below

![](images-readme/pearson.png)

Parameters for the KNN model are:

* K – The (max) number of neighbors to take into account for aggregation
- simoptions – Fixed for this task, pearson similarity
+ bsloptions – Similar to baseline updates and parameters in previous section "BaselineOnly"


### 3.3 Latent Factor Model : SVD 
The utility matrix U is approximated in a lower dimension form as two matrices Q and P. Each representing the low dimensional representation of a user and business. The low dim representation is K = 1 for this task. 

The prediction $\hat{r}_i$ is set as:

$$\hat{r}_{ui} = \mu + b_u + b_i + q_i^Tp_u$$

If user $u$ is unknown, then the bias $b_u$ and the factors $p_u$ are assumed to be zero. The same applies for item ii with $b_i$ and $q_i$.

To estimate all the unknown, we need to minimize the following regularized squared error:
$$
\sum_{r_{ui} \in R_{train}} \left(r_{ui} - \hat{r}_{ui} \right)^2 +
\lambda\left(b_i^2 + b_u^2 + ||q_i||^2 + ||p_u||^2\right)
$$

The minimization is performed by a very straightforward stochastic gradient descent:

\begin{equation}
\begin{split}b_u &\leftarrow b_u + \gamma (e_{ui} - \lambda b_u)\\
b_i &\leftarrow b_i + \gamma (e_{ui} - \lambda b_i)\\
p_u &\leftarrow p_u + \gamma (e_{ui} \cdot q_i - \lambda p_u)\\
q_i &\leftarrow q_i + \gamma (e_{ui} \cdot p_u - \lambda q_i)\end{split}
\end{equation}

where $e_{ui}=r_{ui}$-$\hat{r}_{ui}$ 
These steps are performed over all the ratings of the trainset and repeated $n_{epochs}$ times. Baselines are initialized to $0$. User and item factors are randomly initialized according to a normal distribution

The parameters for SVD are as follows :

* nfactors – The number of factors.
* nepochs – The number of iteration of the SGD procedure
* lrall – The learning rate for all parameters.
* regall – The regularization term for all parameters

### 3.4 Non-Negative Matrix Factorization : NMF 
A collaborative filtering algorithm based on Non-negative Matrix Factorization. This algorithm is very similar to SVD. The prediction $\hat{r}_ui$ is set as:

$$\hat{r}_{ui} = q_i^Tp_u,$$

where user and item factors are kept positive. 
The optimization procedure is a (regularized) stochastic gradient descent with a specific choice of step size that ensures non-negativity of factors, provided that their initial values are also positive.

At each step of the SGD procedure, the factors f or user u and item i are updated as follows:



\begin{equation}
    \begin{split}p_{uf} &\leftarrow p_{uf} \cdot \frac{\sum_{i \in I_u} q_{if}
\cdot r_{ui}}{\sum_{i \in I_u} q_{if} \cdot \hat{r_{ui}} +
\lambda_u |I_u| p_{uf}}\\
q_{if} &\leftarrow q_{if} \cdot \frac{\sum_{u \in U_i} p_{uf}
\cdot r_{ui}}{\sum_{u \in U_i} p_{uf} \cdot \hat{r_{ui}} +
\lambda_i |U_i| q_{if}}\\\end{split}
\end{equation}

where $\lambda_u$ and $\lambda_i$ are regularization parameters
The Parameters for the model are :


* n_factors – The number of factors.
* n_epochs – The number of iteration of the SGD procedure.
* reg_pu – The regularization term for users λu.
* reg_qi – The regularization term for items λi.



### 3.5 Slope One:
A simple yet accurate collaborative filtering algorithm.The prediction $\hat{r}_{ui}$ is set as:

$$\hat{r}_{ui} = \mu_u + \dfrac{1}{
|R_i(u)|}
\sum\limits_{j \in R_i(u)} \text{dev}(i, j)$$


where $R_i(u)$ is the set of relevant items, i.e. the set of items $j$ rated by $u$ that also have at least one common user with $i$. $Dev(i,j)$ is defined as the average difference between the ratings of $i$ and those of $j$:

$$
\text{dev}(i, j) = \dfrac{1}{
|U_{ij}|}\sum\limits_{u \in U_{ij}} r_{ui} - r_{uj}$$


### 3.6 Prediction and Validation

The 6 models presented above are validated independently on the 100,000 validation set to find their respective best parameters. The Baseline ALS algorithm, Latent Factor Model, and KNNBaseline were found to be the best performing out of the three. The predictions were made based on the following update equation

$$ \hat{r}_{ui} = \dfrac{\sum_{k = 1}^{k = 6}w_k * \hat{r}_{uik}}{\sum_{k=1}^{k=6} w_k}$$

where, $w_k$ are the weights associated with each model prediction and $\hat{r}_{uik}$ is the predicted rating of each model. This ensemble model was validated to get the best performing weights $w_k$. The final model was trained using the best parameters and the entire training set (200,000). The weighted average of these models outputs  on the test set were submitted on Kaggle